In [2]:
import numpy as np
import pandas as pd
import os
import struct
import matplotlib.pyplot as plt

# import cv2

import dicom
# import SimpleITK as sitk

Load the available patient images: sample_images.7z from https://www.kaggle.com/c/data-science-bowl-2017/data.

In [3]:
IMAGE_PATH = '../sample_images'
# Each folder in IMAGE_PATH represents one patient scan = one (3D) patient image.
patients = os.listdir(IMAGE_PATH)
patients.sort()
# Each folder contains N DICOM files, where N is equal to the number of slices for the patient.
print(len(os.listdir(IMAGE_PATH + '/' + patients[0])))

134


Read the associated labels.

In [4]:
labels = pd.read_csv('../stage1_labels.csv', index_col=0)
print(labels.head())

,cancer
id,
0015ceb851d7251b8f399e39779d1e7d,1
0030a160d58723ff36d73f41b170ec21,0
003f41c78e6acfa92430a057ac0b306e,0
006b96310a37b36cccb2ab48d10b49a3,1
008464bb8521d09a42985dd8add3d0d2,1


Get the first patient image array. firstPatientImage is a 3D (height x length x breadth) np array.

In [5]:
def getImage(path):
    '''
    Load an image, which consists of multiple slices.
    :param      path: path to a patient image
    :return:    slices: a list of dicom.dataset.FileDataset
    '''
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    return slices

In [8]:
def addThickness(image):
    """
    Update patient image with the thickness of each slice.
    :param      image: a list of dicom.dataset.FileDataset
    :return:    image: a list of dicom.dataset.FileDataset
    """
    image.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        st = np.abs(image[0].ImagePositionPatient[2] - image[1].ImagePositionPatient[2])
    except:
        st = np.abs(image[0].SliceLocation - image[1].SliceLocation)
    for s in image:
        s.SliceThickness = st
    return image

In [6]:
def getImageArray(image):
    '''
    Get scan pixels.
    :param      image: a list of dicom.dataset.FileDataset
    :return:    np.array
    '''
    imageArray = np.stack([x.pixel_array for x in image])
    return imageArray

In [9]:
firstPatient = getImage(IMAGE_PATH + '/' + patients[0])
firstPatient = addThickness(firstPatient)
firstPatientImage = getImageArray(firstPatient)

Show one of the first patient's slices and show the associated histogram of pixel values.

In [ ]:
slice = firstPatientImage[80]
df = pd.DataFrame(slice)
print(df.head())

plt.gray()
plt.imshow(slice, vmin=0, vmax=1000)
plt.show()

plt.hist(slice.flatten(), bins=80)
plt.show()
s = firstPatient[0]
s.RescaleSlope